# "How many token generators do you usually put in a tokens deck, where the commander doesn't make tokens themselves?"

https://www.reddit.com/r/EDH/comments/1poh78x/how_many_token_generators_do_you_usually_put_in_a/

"I'm putting together a [[Fire Lord Zuko]] deck that aims to go wide with tokens and then use his ability to buff all of the tokens.

So I was wondering, how many token generators do you put in go wide decks, when the commander doesn't generate tokens themselves?"

## The Method

1. Download bulk card data from Scryfall.
2. Find commanders.
3. Filter commanders that don't generate tokens.
4. Filter cards that do make tokens.
5. Scrape commander deck data from Moxfield.
6. Filter decks whose commanders don't make tokens.
7. Filter decks whose theme is "Tokens".
8. Calculate average count of cards from step 4 in decks from step 7. 
9. Filter decks whose commander is Fire Lord Zuko.
10. Calculate average count of cards from step 4 in steps from step 9. 

### 1. Download bulk card data from Scryfall.

In [15]:
import requests

resp = requests.get("https://api.scryfall.com/bulk-data")
assert resp.status_code == 200
bulk_data_items = resp.json()
print(bulk_data_items.keys())

dict_keys(['object', 'has_more', 'data'])


In [16]:
for object in bulk_data_items["data"]:
    if object["name"] == "Oracle Cards":
        oracle_cards_url = object["download_uri"]
        break

resp = requests.get(oracle_cards_url)
assert resp.status_code == 200
oracle_cards = resp.json()
print(f"Total Oracle Cards: {len(oracle_cards)}")

Total Oracle Cards: 36708


### 2. Find commanders.

In [17]:
print(oracle_cards[0].keys())
print(oracle_cards[0]["oracle_text"])
resp = requests.get("https://api.scryfall.com/cards/search?q=is%3Acommander")
commanders = resp.json()
print(f"Total Commander Cards: {commanders['total_cards']}")

dict_keys(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'mana_cost', 'cmc', 'type_line', 'oracle_text', 'power', 'toughness', 'colors', 'color_identity', 'keywords', 'all_parts', 'legalities', 'games', 'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'rarity', 'watermark', 'flavor_text', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'frame', 'frame_effects', 'security_stamp', 'full_art', 'textless', 'booster', 'story_spotlight', 'edhrec_rank', 'preview', 'prices', 'related_uris', 'purchase_uris'])
Landfall — Whenever a land you control enters, you get {E}{E} (two energy counters).
You may pay e

In [18]:
# Token keywords that precede a number or "X"
token_keywords_n = [
    "Fabricate",
    "Afterlife",
    "Incubate",
    "Mobilize",
    "Endures"
]

# Token keywords that do not precede a number
token_keywords = [
    "Investigate",
    "Myriad",
    "Offspring",
    "Living weapon",
    "For Mirrodin!",
    "Job select",
    "Embalm",
    "Eternalize",
    "Encore",
    "Populate"
]

# Other token text
other_token_text = [
    "becomes a token",
    "become tokens"
]

In [ ]:
# Build the query
url = "https://api.scryfall.com/cards/search?q="
url_character_codes = {
    ":": "%3A",
    "/": "%2F",
    "[": "%5B",
    "]": "%5D",
    "+": "%2B",
    "\\": "%5C",
    " ": "+"   # Spaces are replaced with plus signs after plus signs are replaced with %2B
}

for token_keyword in token_keywords_n:
    expression = rf"\b{token_keyword}\b [0-9x]+"
    query = f"o:/{expression}/"
    for char, code in url_character_codes.items():
        query = query.replace(char, code)
    print(f"{url}{query}")
    # print(query)

https://api.scryfall.com/cards/search?q=o%3A%2F%5CbFabricate%5Cb+%5B0-9x%5D%2B%2F
https://api.scryfall.com/cards/search?q=o%3A%2F%5CbAfterlife%5Cb+%5B0-9x%5D%2B%2F
https://api.scryfall.com/cards/search?q=o%3A%2F%5CbIncubate%5Cb+%5B0-9x%5D%2B%2F
https://api.scryfall.com/cards/search?q=o%3A%2F%5CbMobilize%5Cb+%5B0-9x%5D%2B%2F
https://api.scryfall.com/cards/search?q=o%3A%2F%5CbEndures%5Cb+%5B0-9x%5D%2B%2F


Amass

Squad

### 3. Filter commanders that don't make tokens.

### 4. Filter cards that do make tokens.

### 5. Scrape commander deck data from Moxfield.

### 6. Filter decks whose commanders don't make tokens.

### 7. Filter decks whose theme is "Tokens".

### 8. Calculate average count of cards from step 4 in decks from step 7.

### 9. Filter decks whose commander is Fire Lord Zuko.

### 10. Calculate average count of cards from step 4 in decks from step 9.